### VERN AdU Waste MobileNetV3 Small Model

In [ ]:
# Install required libraries
!pip install --upgrade pip -q
!pip install tensorflow==2.16.1 tf-keras==2.16.0 scikit-learn==1.4.2 matplotlib seaborn pillow -q
print("Libraries installation completed.")

In [ ]:
# Import necessary libraries
import tensorflow as tf
from tensorflow.keras.applications import MobileNetV3Small
from tensorflow.keras.layers import Dense, GlobalAveragePooling2D, Dropout, BatchNormalization
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.optimizers.schedules import ExponentialDecay
from tensorflow.keras.preprocessing.image import ImageDataGenerator
from tensorflow.keras.callbacks import ModelCheckpoint, EarlyStopping
from tensorflow.keras.regularizers import l2
from tensorflow.keras import backend as K
import numpy as np
import os
import shutil
from sklearn.utils.class_weight import compute_class_weight
from sklearn.metrics import precision_score, recall_score, f1_score, confusion_matrix
import matplotlib.pyplot as plt
import seaborn as sns
from PIL import Image, ImageFile
from google.colab import drive, files

# Allow processing of truncated images
ImageFile.LOAD_TRUNCATED_IMAGES = True
print("Libraries imported successfully.")

In [ ]:
# Mount Google Drive
try:
    drive.mount('/content/drive')
    print("Google Drive mounted successfully.")
except Exception as e:
    print(f"Failed to mount Google Drive: {e}")

# Set paths for dataset and merged dataset
google_drive_path = '/content/drive/MyDrive/CSRP/dataSet'
merged_dataset_dir = '/content/merged_dataset'
os.makedirs(merged_dataset_dir, exist_ok=True)

In [ ]:
# Function to copy and validate images, with enhanced error handling
def copy_and_validate_images(source_dir, subcategories, category, target_dir):
    try:
        target_category_dir = os.path.join(target_dir, category)
        os.makedirs(target_category_dir, exist_ok=True)
        total_images_in_category = 0

        for subcategory in subcategories:
            subcategory_dir = os.path.join(source_dir, category, subcategory)
            image_files = os.listdir(subcategory_dir)
            total_images = len(image_files)
            total_images_in_category += total_images
            print(f"Processing {total_images} images in '{subcategory}' under '{category}'.")

            for img_file in image_files:
                source_file_path = os.path.join(subcategory_dir, img_file)
                target_file_path = os.path.join(target_category_dir, img_file)
                if os.path.exists(target_file_path):
                    continue

                try:
                    with Image.open(source_file_path) as img:
                        if img.mode in ('P', 'PA'):
                            img = img.convert('RGBA')
                        if img.mode == 'RGBA':
                            img = img.convert('RGB')
                        elif img.mode == 'LA':
                            img = img.convert('L')
                        if img.mode not in ('RGB', 'L'):
                            img = img.convert('RGB')
                        img.save(target_file_path)
                except (IOError, OSError) as e:
                    print(f"Skipped image '{img_file}' due to: {e}")

            print(f"Completed '{subcategory}' in '{category}'.")
        print(f"Total images processed in '{category}': {total_images_in_category}\n")
    except Exception as e:
        print(f"Error processing '{category}': {e}")

In [ ]:
# Define subcategories for each waste type
biodegradable_subcategories = ['food_waste', 'leaf_waste', 'paper_waste', 'wood_waste']
non_biodegradable_subcategories = ['ewaste', 'metal_cans', 'other', 'plastic_bags', 'plastic_bottles']
recyclable_subcategories = ['aluminum', 'carton', 'foam_box', 'milk_box', 'other', 'paper', 'paper_cup', 'plastic', 'plastic_cup']

# Copy and validate images
copy_and_validate_images(google_drive_path, biodegradable_subcategories, 'biodegradable', merged_dataset_dir)
copy_and_validate_images(google_drive_path, non_biodegradable_subcategories, 'non_biodegradable', merged_dataset_dir)
copy_and_validate_images(google_drive_path, recyclable_subcategories, 'recyclable', merged_dataset_dir)
print("Dataset merging completed.")

In [ ]:
# Data Augmentation and Preprocessing
train_datagen = ImageDataGenerator(
    rescale=1./255, rotation_range=45, width_shift_range=0.2, height_shift_range=0.2,
    shear_range=0.2, zoom_range=(0.8, 1.2), horizontal_flip=True, vertical_flip=True,
    brightness_range=[0.7, 1.3], fill_mode='nearest', validation_split=0.2
)
test_datagen = ImageDataGenerator(rescale=1./255)

# Create train, validation, and test generators
train_generator = train_datagen.flow_from_directory(
    merged_dataset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='training'
)
validation_generator = train_datagen.flow_from_directory(
    merged_dataset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical', subset='validation'
)
test_generator = test_datagen.flow_from_directory(
    merged_dataset_dir, target_size=(224, 224), batch_size=32, class_mode='categorical'
)

print(f"Training samples: {train_generator.samples}, Validation samples: {validation_generator.samples}")

In [ ]:
# Compute class weights to handle class imbalance
try:
    class_weights = compute_class_weight(
        class_weight='balanced', classes=np.unique(train_generator.classes), y=train_generator.classes
    )
    class_weights = dict(enumerate(class_weights))
    print("Class weights calculated.")
except Exception as e:
    print(f"Class weight calculation error: {e}")

In [ ]:
# Model building with improved architecture
try:
    base_model = MobileNetV3Small(input_shape=(224, 224, 3), include_top=False, weights='imagenet')
    num_layers_to_freeze = int(0.05 * len(base_model.layers))
    for layer in base_model.layers[:num_layers_to_freeze]:
        layer.trainable = False

    x = GlobalAveragePooling2D()(base_model.output)
    x = BatchNormalization()(x)
    x = Dense(512, activation='relu', kernel_regularizer=l2(1e-4))(x)
    x = Dropout(0.4)(x)
    predictions = Dense(train_generator.num_classes, activation='softmax')(x)
    model = Model(inputs=base_model.input, outputs=predictions)

    initial_lr = 1e-4
    lr_schedule = ExponentialDecay(initial_learning_rate=initial_lr, decay_steps=5 * len(train_generator), decay_rate=0.8, staircase=True)
    model.compile(optimizer=Adam(learning_rate=lr_schedule), loss='categorical_crossentropy', metrics=['accuracy'])
    print("Model built and compiled.")
except Exception as e:
    print(f"Model building or compilation error: {e}")

In [ ]:
# Model training with enhanced validation and callbacks
try:
    history = model.fit(
        train_generator, epochs=15, validation_data=validation_generator, class_weight=class_weights,
        callbacks=[
            ModelCheckpoint('best_model.keras', monitor='val_accuracy', save_best_only=True, mode='max'),
            EarlyStopping(monitor='val_loss', patience=5, restore_best_weights=True)
        ]
    )
    print("Model training completed.")
except Exception as e:
    print(f"Model training error: {e}")

In [ ]:
# Model evaluation and performance metrics
try:
    test_loss, test_accuracy = model.evaluate(test_generator)
    print(f"Test Loss: {test_loss}, Test Accuracy: {test_accuracy}")

    y_true = test_generator.classes
    y_pred_classes = np.argmax(model.predict(test_generator), axis=1)
    precision = precision_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    recall = recall_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    f1 = f1_score(y_true, y_pred_classes, average='weighted', zero_division=0)
    print(f"Precision: {precision}, Recall: {recall}, F1 Score: {f1}")
except Exception as e:
    print(f"Model evaluation error: {e}")

In [ ]:
# Plot training history for accuracy and loss
try:
    plt.figure(figsize=(12, 6))
    plt.subplot(1, 2, 1)
    plt.plot(history.history['accuracy'], label='Train Accuracy')
    plt.plot(history.history['val_accuracy'], label='Validation Accuracy')
    plt.title('Model Accuracy')
    plt.xlabel('Epoch')
    plt.ylabel('Accuracy')
    plt.legend(loc='upper left')

    plt.subplot(1, 2, 2)
    plt.plot(history.history['loss'], label='Train Loss')
    plt.plot(history.history['val_loss'], label='Validation Loss')
    plt.title('Model Loss')
    plt.xlabel('Epoch')
    plt.ylabel('Loss')
    plt.legend(loc='upper right')
    plt.tight_layout()
    plt.show()
    print("Training history plotted.")
except Exception as e:
    print(f"Plotting error: {e}")

In [ ]:
# Generate and display confusion matrix
try:
    conf_matrix = confusion_matrix(y_true, y_pred_classes)
    plt.figure(figsize=(8, 6))
    sns.heatmap(conf_matrix, annot=True, fmt='d', cmap='Blues',
                xticklabels=test_generator.class_indices, yticklabels=test_generator.class_indices)
    plt.xlabel('Predicted')
    plt.ylabel('Actual')
    plt.title('Confusion Matrix')
    plt.show()
    print("Confusion matrix displayed.")
except Exception as e:
    print(f"Confusion matrix error: {e}")

In [ ]:
# Clear TensorFlow session and convert to TensorFlow Lite model for deployment
K.clear_session()

try:
    model.export('saved_model')
    converter = tf.lite.TFLiteConverter.from_saved_model('saved_model')
    converter.optimizations = [tf.lite.Optimize.DEFAULT]
    converter.target_spec.supported_types = [tf.float16]
    converter.target_spec.supported_ops = [tf.lite.OpsSet.TFLITE_BUILTINS, tf.lite.OpsSet.SELECT_TF_OPS]
    tflite_model = converter.convert()

    # Save and download the TensorFlow Lite model
    with open('waste_classifier.tflite', 'wb') as f:
        f.write(tflite_model)
    print("Model successfully converted to TensorFlow Lite.")
    files.download('waste_classifier.tflite')
    print("TensorFlow Lite model downloaded.")
except Exception as e:
    print(f"TensorFlow Lite conversion error: {e}")